In [9]:
import numpy as np
import pandas as pd
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, time


In [10]:
def preprocess(path, alpha=0.0001):
    df = pd.read_csv(path)

    if "m-" in df.columns:
        df = df.drop(["m-", "m+", "l(3)", "l(4)"], axis=1)

    # mean prices and volumes*
    
    df['mean_sp'] = df.filter(like='sp').mean(axis=1)
    df['mean_bp'] = df.filter(like='bp').mean(axis=1)
    df['mean_sq'] = df.filter(like='sq').mean(axis=1)
    df['mean_bq'] = df.filter(like='bq').mean(axis=1)
    
    for i in range(1,6):
        # bid-ask spreads and mid-prices
        df[f'bid-ask-spread{i}'] = df[f'sp{i}'] - df[f'bp{i}']
        df[f'mid-price{i}'] = (df[f'sp{i}'] + df[f'bp{i}'])/2

        # accumulated differences
        df['accumulated_price_difference'] = df[f'bid-ask-spread{i}'] if i == 1 else df['accumulated_price_difference'] + df[f'bid-ask-spread{i}']
        df['accumulated_quantity_difference'] = (df[f'sq{i}'] - df[f'bq{i}']) if i == 1 else df['accumulated_price_difference'] + ((df[f'sq{i}'] - df[f'bq{i}']))
        # price differences
        df['sp_difference' + (f'{1}' if i == 5 else f'{i+1}') + f'{i}'] = abs(df['sp1'] - df[f'sp{i}']) if i == 5 else abs(df[f'sp{i+1}'] - df[f'sp{i}'])
        df['bp_difference' + (f'{1}' if i == 5 else f'{i+1}') + f'{i}'] = abs(df['bp1'] - df[f'bp{i}']) if i == 5 else abs(df[f'bp{i+1}'] - df[f'bp{i}'])
        
        # price and volume derivatives
        df[f'sp_derviative{i}'] = df[f'sp{i}'] - df[f'sp{i}'].shift(1)
        df[f'bp_derviative{i}'] = df[f'bp{i}'] - df[f'bp{i}'].shift(1)
        df[f'sq_derviative{i}'] = df[f'sq{i}'] - df[f'sq{i}'].shift(1)
        df[f'bq_derviative{i}'] = df[f'bq{i}'] - df[f'bq{i}'].shift(1)
    
    # output label calculation
    for k in range(5, 35, 5):
        m = df['mid-price1'].rolling(window=k).mean()
        M = df['mid-price1'].shift(-k).rolling(window=k).mean()
        l = (m - df['mid-price1'])/df['mid-price1']
        L = (m - M)/M
        df[f'y{k}(3)'] = 0
        df.loc[l > alpha, f'y{k}(3)'] = 1
        df.loc[l < -alpha, f'y{k}(3)'] = -1
        df[f'y{k}(4)'] = 0
        df.loc[L > alpha, f'y{k}(4)'] = 1
        df.loc[L < -alpha, f'y{k}(4)'] = -1

    df.to_csv(path, index=False)


In [11]:

files = ["D:\projects\HFT\ALUMINIUM23FEBFUT\output.csv"]

for file in files:
    preprocess(file)



In [12]:
def trim(path):
    df = pd.read_csv(path)
    # Convert the timestamp column to datetime format
    df['time'] = pd.to_datetime(df['date'] + " " + df['time'])

    # Define the start and end times
    start_time = time(9, 31, 0) #type: ignore
    end_time = time(15, 19, 0) #type: ignore

    # Filter the dataframe to include only entries within the specified time range
    mask = (df['time'].dt.time >= start_time) & (
        df['time'].dt.time <= end_time)
    df = df.loc[mask]

    df.to_csv(path, index=False)


In [13]:
files = ["D:\projects\HFT\ALUMINIUM23FEBFUT\output.csv"]

for file in files:
    trim(file)


In [14]:
d = {}

In [15]:
# root = "./DATA New/"
# df = pd.read_csv("./countStats.csv")
# selected_files = list(df['name'])
# folders = [f.path for f in os.scandir(root) if f.is_dir()]
# files = []

# for folder in folders:
#     subfolders = [f.path for f in os.scandir(folder) if f.is_dir()]
#     files += [f.path for f in os.scandir(folder) if f.is_file()]
#     for subfolder in subfolders:
#         files += [f.path for f in os.scandir(subfolder) if f.is_file()]
# for file in files:
#     if file.split("/")[-1].split("\\")[-1].split(".")[0] in selected_files:
#         x = file.split("\\")[-1].split(".")[0]
#         if x in d:
#             d[x].append(file)
#         else:
#             d[x] = [file]

In [16]:
# count = 0
# for i in d:
#     count += 1  
#     df = pd.read_csv(d[i][0])
#     for p in range(1, len(d[i])):
#         count += 1
#         x = pd.read_csv(d[i][p])
#         df = pd.concat([df, x])
#     if 'BANKNIFTY' in i:
#         s = 'BANKNIFTY'
#     else:
#         s = 'NIFTY'
#     df.to_csv(f"./final_new_data/{s}/{i}.csv", index=False)
#     print(f'file {count} processed!')
